In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

In [ ]:
plot_series(
    df=df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
)

## Zero-shot forecasting

In [ ]:
from dotenv import load_dotenv
from nixtla import NixtlaClient

In [ ]:
# Initialize NixtlaClient


In [ ]:
HORIZON = 13

In [ ]:
# Zero-shot forecasting

fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
from functools import reduce

In [ ]:
# Define default column names

# List all possible models

# List to store all cross-validation results

for model in models:
    # Run cross-validation

    # Rename columns
    
    # Append results

# Merge all results

final_cv_df.head()

In [ ]:
eval_df = evaluate(
    final_cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    level=[80],
    agg_fn="mean"
)
eval_df

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df[["Store", "Date", "timegpt-2.1", "timegpt-2.1-lo-80", "timegpt-2.1-hi-80"]],
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Exogenous features

In [ ]:
df.head()

In [ ]:
exog_cv_df = client.cross_validation(
    df=df,
    h=HORIZON,
    n_windows=3,
    step_size=HORIZON,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    # Specify historical exogenous features

    model="timegpt-2.1"
)
exog_cv_df = exog_cv_df.rename(columns={"TimeGPT": "timegpt-2.1"})

eval_df = evaluate(
    exog_cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

In [ ]:
exog_cv_df = client.cross_validation(
    # Drop historical exogenous features

    h=HORIZON,
    n_windows=3,
    step_size=HORIZON,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    model="timegpt-2.1"
)
exog_cv_df = exog_cv_df.rename(columns={"TimeGPT": "timegpt-2.1"})

eval_df = evaluate(
    exog_cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

### Explainability

In [ ]:
test_df = df.groupby("Store").tail(13)
input_df = df.drop(test_df.index)
X_df = test_df[["Store", "Date", "Holiday_Flag"]]

In [ ]:
exog_fcsts_df = client.forecast(
    df=input_df,
    X_df=X_df,
    h=HORIZON,
    freq="W-FRI",
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    model="timegpt-2.1",
    # Get feature contributions

)
exog_fcsts_df.head()

In [ ]:
# Access shap values

shap_df.head()

In [ ]:
test_df.head(13)

In [ ]:
import shap

# Get feature columns for SHAP values

# Create shap.Explanation object

# Make a bar plot


In [ ]:
selected_ds = "2012-09-07"
selected_store = 1

# Filter shap_df
filtered_shap_df = shap_df[(shap_df["Date"] == selected_ds) & (shap_df["Store"] == selected_store)]

# Create shap.Explanation object

# Make a waterfall plot

plt.title(f"Waterfall plot for Store {selected_store} on {selected_ds}")
plt.show()

## Multivariate forecasting

In [ ]:
mv_cv_df = client.cross_validation(
    df=df,
    h=HORIZON,
    n_windows=3,
    step_size=HORIZON,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    hist_exog_list=["Temperature", "Fuel_Price", "CPI", "Unemployment"],
    model="timegpt-2.1",
    # Use multivariate forecasting

)
mv_cv_df = mv_cv_df.rename(columns={"TimeGPT": "timegpt-2.1"})

eval_df = evaluate(
    mv_cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Fine-tuning

In [ ]:
ft_cv_df = client.cross_validation(
    df=df,
    h=HORIZON,
    n_windows=3,
    step_size=HORIZON,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    hist_exog_list=["Temperature", "Fuel_Price", "CPI", "Unemployment"],
    model="timegpt-2.1",
    multivariate=True,
    # Set finetune_steps

    # Set finetune_loss

)
ft_cv_df = ft_cv_df.rename(columns={"TimeGPT": "timegpt-2.1"})

eval_df = evaluate(
    ft_cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df